In [1]:
import pandas as pd 
import numpy as np
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import plotly_express as px
from geopy.point import Point
from tqdm import tqdm
#from pandas_profiling import ProfileReport

## Import df from src 
filename = 'src/data_100000.csv'
df = pd.read_csv(filename)
#filename = 'missing_location.csv'
df2 = pd.read_csv(filename)
#Renaming col
df = df.rename(columns={"vehicle_type_code1": "vehicle_type_code_1", "vehicle_type_code2": "vehicle_type_code_2"})
#df2
#profile = ProfileReport(df, title="Pandas Profiling Report")

In [2]:
## Check for missing,nul,NaN values
print("There are {} rows of data".format(len(df)))
nul_data = df.isnull().sum()
## setting to false missing data
nul_data

There are 100000 rows of data


crash_date                           0
crash_time                           0
borough                          35026
zip_code                         35034
latitude                          8035
longitude                         8035
location                          8035
on_street_name                   26009
off_street_name                  52875
cross_street_name                74033
number_of_persons_injured            0
number_of_persons_killed             0
number_of_pedestrians_injured        0
number_of_pedestrians_killed         0
number_of_cyclist_injured            0
number_of_cyclist_killed             0
number_of_motorist_injured           0
number_of_motorist_killed            0
contributing_factor_vehicle_1      371
contributing_factor_vehicle_2    19243
contributing_factor_vehicle_3    91239
contributing_factor_vehicle_4    97760
contributing_factor_vehicle_5    99333
collision_id                         0
vehicle_type_code_1                740
vehicle_type_code_2      

In [3]:
vehicle_type_code = ['vehicle_type_code_1','vehicle_type_code_2','vehicle_type_code_3','vehicle_type_code_4','vehicle_type_code_5']
nb_accident_by_vehicle = df[['vehicle_type_code_1','vehicle_type_code_2','vehicle_type_code_3','vehicle_type_code_4','vehicle_type_code_5']].apply(pd.value_counts)
df[vehicle_type_code] = df[vehicle_type_code].fillna('Unspecified')
df[vehicle_type_code]

,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,Station Wagon/Sport Utility Vehicle,Unspecified,Unspecified,Unspecified,Unspecified
1,Sedan,Unspecified,Unspecified,Unspecified,Unspecified
2,Sedan,Sedan,Unspecified,Unspecified,Unspecified
3,Motorcycle,Sedan,Bus,Unspecified,Unspecified
4,Sedan,Station Wagon/Sport Utility Vehicle,Unspecified,Unspecified,Unspecified
...,...,...,...,...,...
99995,Station Wagon/Sport Utility Vehicle,Unspecified,Unspecified,Unspecified,Unspecified
99996,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,Unspecified,Unspecified,Unspecified
99997,Sedan,Box Truck,Unspecified,Unspecified,Unspecified
99998,Sedan,Bus,Unspecified,Unspecified,Unspecified


In [4]:
contributing_factor = ['contributing_factor_vehicle_1','contributing_factor_vehicle_2','contributing_factor_vehicle_3','contributing_factor_vehicle_4','contributing_factor_vehicle_5']
nb_accident_by_factor = df[['contributing_factor_vehicle_1','contributing_factor_vehicle_2','contributing_factor_vehicle_3','contributing_factor_vehicle_4','contributing_factor_vehicle_5']].apply(pd.value_counts)
df[contributing_factor]= df[contributing_factor].fillna('Unspecidifed')

In [5]:
df.sort_values(by='location',inplace=True)
df.zip_code = df.zip_code.fillna(method='ffill')
df.borough = df.borough.fillna(method='ffill')
## Might be wrong need the make a if to check if off or on are null then fillna
df.cross_street_name = np.where(df['on_street_name'].isnull(),df['cross_street_name'].fillna(method='ffill'),False)
df.on_street_name = df.on_street_name.fillna(method='ffill')
df.off_street_name = df.off_street_name.fillna(method='ffill')

In [7]:
import time
locator = Nominatim(user_agent='myGeocoder')
df['on_street_name'] = df['on_street_name'].apply(lambda x: x.strip())
df['borough'] = df['borough'].apply(lambda x: x.strip())
df['adress']= df['off_street_name']+','+df['borough']+','+'United States of America'
empty_location = df[(df['location'].isna()) | (df['location'] == '(0.0, 0.0)')]
for x in empty_location.index:
    location = locator.geocode(df['adress'][x],timeout=10)
    if location is None:
        empty_location.at[x,'latitude']= None
        empty_location.at[x,'longitude']= None
    else:
        print(x)
        print(location)
        print(location.latitude)
        empty_location.at[x,'latitude']= location.latitude
        empty_location.at[x,'longitude']= location.longitude
        empty_location.at[x,'location']= str(location.latitude)+','+str(location.longitude)       
empty_location.to_csv('updated_location.csv')


,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,cross_street_name,...,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,adress
91845,2020-03-28T00:00:00.000,18:50,BROOKLYN,11233.0,0.0,0.0,"(0.0, 0.0)",ATLANTIC AVENUE,SACKMAN STREET,False,...,Unspecidifed,Unspecidifed,Unspecidifed,4304383,Sedan,Sedan,Unspecified,Unspecified,Unspecified,"SACKMAN STREET,BROOKLYN,United States of America"
70102,2020-06-16T00:00:00.000,8:11,QUEENS,11104.0,0.0,0.0,"(0.0, 0.0)",43 STREET,QUEENS BOULEVARD,False,...,Unspecidifed,Unspecidifed,Unspecidifed,4320537,Station Wagon/Sport Utility Vehicle,Sedan,Unspecified,Unspecified,Unspecified,"QUEENS BOULEVARD,QUEENS,United States of America"
49443,2020-09-29T00:00:00.000,3:20,BROOKLYN,11209.0,0.0,0.0,"(0.0, 0.0)",43 STREET,QUEENS BOULEVARD,339 BAY RIDGE PARKWAY,...,Unspecidifed,Unspecidifed,Unspecidifed,4352797,Sedan,Sedan,Unspecified,Unspecified,Unspecified,"QUEENS BOULEVARD,BROOKLYN,United States of Ame..."
49549,2020-09-13T00:00:00.000,14:25,BRONX,10468.0,0.0,0.0,"(0.0, 0.0)",EAST BEDFORD PARK BOULEVARD,VILLA AVENUE,False,...,Unspecidifed,Unspecidifed,Unspecidifed,4347357,Station Wagon/Sport Utility Vehicle,Unspecified,Unspecified,Unspecified,Unspecified,"VILLA AVENUE,BRONX,United States of America"
49584,2020-09-28T00:00:00.000,0:00,QUEENS,11412.0,0.0,0.0,"(0.0, 0.0)",115 AVENUE,200 STREET,False,...,Unspecidifed,Unspecidifed,Unspecidifed,4352807,Sedan,Sedan,Sedan,Unspecified,Unspecified,"200 STREET,QUEENS,United States of America"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99935,2019-11-21T00:00:00.000,17:00,BRONX,10467.0,NaN,NaN,NaN,CROSS ISLAND PARKWAY,149 STREET,False,...,Unspecidifed,Unspecidifed,Unspecidifed,4245413,Sedan,Unspecified,Unspecified,Unspecified,Unspecified,"149 STREET,BRONX,United States of America"
99944,2019-11-15T00:00:00.000,13:52,QUEENS,11373.0,NaN,NaN,NaN,HORACE HARDING EXPRESSWAY,QUEENS BOULEVARD,False,...,Unspecidifed,Unspecidifed,Unspecidifed,4241598,Station Wagon/Sport Utility Vehicle,Box Truck,Unspecified,Unspecified,Unspecified,"QUEENS BOULEVARD,QUEENS,United States of America"
99965,2020-03-04T00:00:00.000,23:47,QUEENS,11373.0,NaN,NaN,NaN,WAKEFIELD AVENUE,BULLARD AVENUE,False,...,Unspecified,Unspecidifed,Unspecidifed,4297737,Sedan,Pick-up Truck,Sedan,Unspecified,Unspecified,"BULLARD AVENUE,QUEENS,United States of America"
99982,2019-11-07T00:00:00.000,17:08,QUEENS,11373.0,NaN,NaN,NaN,BRONX WHITESTONE BRIDGE,BULLARD AVENUE,False,...,Unspecidifed,Unspecidifed,Unspecidifed,4237898,Station Wagon/Sport Utility Vehicle,Sedan,Unspecified,Unspecified,Unspecified,"BULLARD AVENUE,QUEENS,United States of America"
